<a href="https://colab.research.google.com/github/Geetika2282/Deep-Learning/blob/main/IMDB_LSTM_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [64]:
!pip install kaggle

Importing the dependencies

In [87]:
import os
import json

from zipfile import ZipFile
import pandas as pd
from tensorflow.keras.layers import Dense, Embedding, LSTM
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential

Data collection from kaggle API

In [88]:
kaggle_dictionary = json.load(open("/content/drive/MyDrive/Datasets/kaggle.json"))

In [89]:
# Setup kaggle credentials as environment varibles
os.environ["KAGGLE_USERNAME"] = kaggle_dictionary["username"]
os.environ["KAGGLE_KEY"] = kaggle_dictionary["key"]

In [90]:
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

Dataset URL: https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
License(s): other
imdb-dataset-of-50k-movie-reviews.zip: Skipping, found more recently modified local copy (use --force to force download)


In [91]:
!ls

 drive	'IMDB Dataset.csv'   imdb-dataset-of-50k-movie-reviews.zip   sample_data


In [92]:
# Unzip the dataset file
with ZipFile("imdb-dataset-of-50k-movie-reviews.zip", "r") as zip_file:
  zip_file.extractall()

In [93]:
!ls

 drive	'IMDB Dataset.csv'   imdb-dataset-of-50k-movie-reviews.zip   sample_data


Loading the dataset

In [94]:
df = pd.read_csv("IMDB Dataset.csv")

In [95]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [96]:
df['sentiment'].value_counts()

,count
sentiment,
positive,25000
negative,25000


Replacing positive with 1, and negative with 0

In [97]:
df.replace({"sentiment": {"positive": 1, "negative": 0}}, inplace=True)

<ipython-input-97-0554edc9e970>:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace({"sentiment": {"positive": 1, "negative": 0}}, inplace=True)


In [98]:
df['sentiment'].value_counts()

,count
sentiment,
1,25000
0,25000


### Split in train_test

In [99]:
train, test = train_test_split(df, test_size=0.2, random_state=24)

In [100]:
train.shape, test.shape

((40000, 2), (10000, 2))

### Preproceesing data

In [102]:
# Tokenize the data - to convert words to vectors/ int
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(train["review"])
X_train = pad_sequences(tokenizer.texts_to_sequences(train["review"]), maxlen=200)
X_test = pad_sequences(tokenizer.texts_to_sequences(test["review"]), maxlen=200)

In [103]:
X_test

array([[   3,    7,    7, ...,    6,   54,  970],
       [   0,    0,    0, ...,  787,   11,   19],
       [   0,    0,    0, ...,  512,    5, 1740],
       ...,
       [   0,    0,    0, ...,   12,   74,   61],
       [   2,  510,   18, ...,    3,  338,   56],
       [   0,    0,    0, ...,  510,   37,   11]], dtype=int32)

In [104]:
y_train = train['sentiment']
y_test = test['sentiment']

In [105]:
y_train

,sentiment
31724,1
9389,1
17228,1
41843,0
31584,0
...,...
21633,0
19857,1
14528,1
899,0


In [84]:
y_test

,sentiment
11022,1
7940,0
30599,0
12942,1
32515,1
...,...
25599,1
5165,0
10231,0
13642,1


# LSTM

In [106]:
# Building the model

model = Sequential()
# Embedding layer should always be the 1st one when working on NLP related problems
# input_dim = 5000 coming from tokenizer number of words in vocab
# input_len = 200; as maxlen = 200

# build the model

model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=128, input_length=200))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation="sigmoid"))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [107]:
model.summary()

Model: "sequential_10"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_8 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_8 (LSTM)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_8 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)